In [ ]:
%reload_kedro
print(catalog.list())

In [1]:
# load dataframe for feature analysis
combined_data = catalog.load("merge_who_ears_owid_data")

[01/09/23 08:12:44] INFO     Loading data from 'merge_who_ears_owid_data' (ParquetDataSet)...   ]8;id=778274;file:///usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=550300;file:///usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/kedro/io/data_catalog.py#343\343]8;;\

In [7]:
# install pycaret as a way to do some quick analysis of all regression model types to know which performs the best 
!pip install --pre pycaret
!pip install xgboost
# !pip install --force-reinstall scikit-learn=='0.23.2' 
!pip install folium
!pip install dtale
# For mac you will have to do a conda install for lightgbm
import sys
!conda install --yes --prefix {sys.prefix} -c conda-forge lightgbm

Solving environment: done

# All requested packages already installed.



In [8]:
from pycaret.regression import *

# importing the required libraries
import pandas as pd
import numpy as np
# Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium 
from folium import plugins

# Manipulating the default plot size
plt.rcParams['figure.figsize'] = 10, 12

# Disable warnings 
import warnings
warnings.filterwarnings('ignore')

In [9]:
combined_data.head()

,date,the_cause_of_the_virus,covid_19_variants,demographic_vulnerability_and_risks,impact_on_mental_health,current_treatments,covid_19_vaccine,health_care_workers_and_vaccine,general_vaccine_discussion,science_and_r_and_d,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,2020-12-15,0,4,0,0,2,1,0,1,0,...,NaN,26.664,NaN,61.15,0.581,35588996.0,NaN,NaN,NaN,NaN
1,2020-12-16,0,1,2,1,0,1,2,0,0,...,NaN,26.664,NaN,61.15,0.581,35588996.0,NaN,NaN,NaN,NaN
2,2020-12-17,1,1,1,0,1,1,0,0,1,...,NaN,26.664,NaN,61.15,0.581,35588996.0,NaN,NaN,NaN,NaN
3,2020-12-18,1,7,0,0,1,3,1,0,2,...,NaN,26.664,NaN,61.15,0.581,35588996.0,NaN,NaN,NaN,NaN
4,2020-12-19,0,6,3,0,1,1,1,0,0,...,NaN,26.664,NaN,61.15,0.581,35588996.0,NaN,NaN,NaN,NaN


In [10]:
combined_data.dtypes

date                                       datetime64[ns]
the_cause_of_the_virus                              int64
covid_19_variants                                   int64
demographic_vulnerability_and_risks                 int64
impact_on_mental_health                             int64
                                                ...      
population                                        float64
excess_mortality_cumulative_absolute              float64
excess_mortality_cumulative                       float64
excess_mortality                                  float64
excess_mortality_cumulative_per_million           float64
Length: 356, dtype: object

In [11]:
# First we need to update our date column to a standard datetime object in python
combined_data['date'] = pd.to_datetime(combined_data['date'], format = '%Y-%m-%d')


In [12]:
combined_data.dtypes

date                                       datetime64[ns]
the_cause_of_the_virus                              int64
covid_19_variants                                   int64
demographic_vulnerability_and_risks                 int64
impact_on_mental_health                             int64
                                                ...      
population                                        float64
excess_mortality_cumulative_absolute              float64
excess_mortality_cumulative                       float64
excess_mortality                                  float64
excess_mortality_cumulative_per_million           float64
Length: 356, dtype: object

In [13]:
# lets next create a sensible feature set for training and testing 
features = combined_data[['date', 'mis_and_disinformation', 'mis_and_disinformation_complaints', 
                          'mis_and_disinformation_delta', 'mis_and_disinformation_male', 
                          'mis_and_disinformation_percent', 'mis_and_disinformation_questions', 
                          'mis_and_disinformation_female',
                         'myths', 'myths_complaints', 'myths_delta', 'myths_female', 
                          'myths_male', 'myths_percent', 'myths_questions', 'total_cases']]
features = features.dropna()


In [14]:
features.head

<bound method NDFrame.head of             date  mis_and_disinformation  mis_and_disinformation_complaints  \
0     2020-12-15                       0                                  0   
1     2020-12-16                       1                                  1   
2     2020-12-17                       2                                  0   
3     2020-12-18                       2                                  1   
4     2020-12-19                       0                                  0   
...          ...                     ...                                ...   
22585 2023-01-02                      43                                 15   
22586 2023-01-03                      62                                 20   
22587 2023-01-04                      45                                 17   
22588 2023-01-05                      44                                 18   
22589 2023-01-06                      44                                 15   

       mis_and_disinformation_delta  mis_and_disinformation_male  \
0                              0.00                            0   
1                            100.00                            0   
2                            100.00                            0   
3                              0.00                            0   
4                           -100.00                            0   
...                             ...                          ...   
22585                         26.47                           17   
22586                         44.19                           38   
22587                        -27.42                           20   
22588                         -2.22                           27   
22589                          0.00                           20   

       mis_and_disinformation_percent  mis_and_disinformation_questions  \
0                                0.00                                 0   
1                                3.70                                 0   
2                                6.90                                 0   
3                                3.39                                 0   
4                                0.00                                 0   
...                               ...                               ...   
22585                            4.29                                 8   
22586                            4.72                                 7   
22587                            4.34                                 9   
22588                            4.31                                 5   
22589                            3.54                                 7   

       mis_and_disinformation_female  myths  myths_complaints  myths_delta  \
0                                  0      1                 0         0.00   
1                                  0      1                 1         0.00   
2                                  0      0                 0      -100.00   
3                                  0      0                 0       100.00   
4                                  0      1                 1       100.00   
...                              ...    ...               ...          ...   
22585                              6      9                 6       350.00   
22586                             10      7                 0       -22.22   
22587                             10     10                 0        42.86   
22588                              3     14                 3        40.00   
22589                              9      4                 0       -71.43   

       myths_female  myths_male  myths_percent  myths_questions  total_cases  
0                 0           1           4.35                0      16362.0  
1                 0           1           3.70                0      16407.0  
2                 0           0           0.00                0      16484.0  
3                 0           0           0.00    

In [16]:
# lets examine the feature set 
import dtale
dtale.show(features)

<class 'dtale.views.DtaleData'>.__repr__ returned empty string

In [17]:
# lets setup our first session for eval
# we chose total cases since its a complete set AND is has high correlation with the other features (Pearson)
session_1 = setup(features, target = 'total_cases', 
                  session_id=12, 
                  log_experiment=False, 
                  experiment_name='total_cases_1')


,Description,Value
0,Session id,12
1,Target,total_cases
2,Target type,Regression
3,Data shape,"(22590, 18)"
4,Train data shape,"(15812, 18)"
5,Test data shape,"(6778, 18)"
6,Numeric features,14
7,Date features,1
8,Preprocess,True
9,Imputation type,simple


In [18]:
# We can now compare all the regression models available 
best_model = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,2055285.1615,21414459953348.4062,4605871.7372,0.8943,1.6438,7.9727,0.6300
lightgbm,Light Gradient Boosting Machine,2170889.0113,22750696470213.8398,4746443.9392,0.8881,1.6900,8.5670,0.0580
rf,Random Forest Regressor,2082343.7888,24123605687197.2227,4887715.0118,0.8812,1.6150,7.9742,1.0710
xgboost,Extreme Gradient Boosting,2323990.4000,24692571452211.1992,4953127.3500,0.8787,1.7136,8.5501,0.3640
gbr,Gradient Boosting Regressor,2976608.0074,33417836066489.5469,5766459.7292,0.8354,1.8673,12.1913,0.3200
dt,Decision Tree Regressor,2276023.9195,43162202407994.0234,6540403.1190,0.7873,1.8408,7.8714,0.0300
knn,K Neighbors Regressor,3706816.4250,58931718127616.0000,7661783.3000,0.7094,1.9783,12.8195,0.0310
ada,AdaBoost Regressor,5348634.9720,78942965456947.6094,8879935.3961,0.6092,2.4033,32.7572,0.0570
llar,Lasso Least Angle Regression,5753289.9304,106042991780581.6562,10283341.1743,0.4764,2.4883,34.4840,0.0160
ridge,Ridge Regression,5753155.3094,106057493778721.8438,10284000.9602,0.4763,2.4881,34.4784,0.0160


Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/usr/local/anaconda3/

2023-01-09 08:24:11,498 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    c

Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    ctx.run(
  File "/usr/local/anaconda3/

2023-01-09 08:24:19,305 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    c

...

2023-01-09 08:25:13,171 - ERROR    - Exception on /dtale/charts/_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/flask/app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dash/dash.py", line 1274, in dispatch
    c

Exception occurred while processing request: Must provide 'func' or tuples of '(column, aggfunc).
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 111, in _handle_exceptions
    return func(*args, **kwargs)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 1779, in reshape_data
    instance = startup(data=builder.reshape(), ignore_duplicate=True)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 48, in reshape
    return self.builder.reshape(data)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 62, in reshape
    pivot_data = pd.pivot_table(
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/pandas/core/reshape/pivot.py", line 97, in pivot_table
    tabl

2023-01-09 08:25:55,799 - ERROR    - Exception occurred while processing request: Must provide 'func' or tuples of '(column, aggfunc).
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 111, in _handle_exceptions
    return func(*args, **kwargs)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 1779, in reshape_data
    instance = startup(data=builder.reshape(), ignore_duplicate=True)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 48, in reshape
    return self.builder.reshape(data)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 62, in reshape
    pivot_data = pd.pivot_table(
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/pandas/core/reshape/pivot.

Exception occurred while processing request: Must provide 'func' or tuples of '(column, aggfunc).
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 111, in _handle_exceptions
    return func(*args, **kwargs)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 1779, in reshape_data
    instance = startup(data=builder.reshape(), ignore_duplicate=True)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 48, in reshape
    return self.builder.reshape(data)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 62, in reshape
    pivot_data = pd.pivot_table(
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/pandas/core/reshape/pivot.py", line 97, in pivot_table
    tabl

2023-01-09 08:26:31,340 - ERROR    - Exception occurred while processing request: Must provide 'func' or tuples of '(column, aggfunc).
Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 111, in _handle_exceptions
    return func(*args, **kwargs)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/views.py", line 1779, in reshape_data
    instance = startup(data=builder.reshape(), ignore_duplicate=True)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 48, in reshape
    return self.builder.reshape(data)
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/dtale/data_reshapers.py", line 62, in reshape
    pivot_data = pd.pivot_table(
  File "/usr/local/anaconda3/envs/who-ears-social-listening/lib/python3.10/site-packages/pandas/core/reshape/pivot.